# Using SyGNet

This notebook demonstrates the basic functionality of the **sygnet** package in Python.

To download the package, simply run `pip install sygnet` at the command line.

## Prerequisites

To start, we load the required packages and define a data generating process (DGP).

In [1]:
%%capture
%cd ..

import pandas as pd
import numpy as np
from numpy.random import default_rng

# NB: once installed via pip, can run `from sygnet import SygnetModel`
from src.sygnet.sygnet_interface import SygnetModel


Next, we will focus on a very simple case of learning a parametric relationship between numeric variables.The DGP comprises two random uniform variables (`x1` and `x2`), which in turn impact the value of two other variables (`x3` and `y`). Once defined, we take 100,000 draws from this DGP to use as our training data:

In [2]:
rng = default_rng()

def gen_sim_data(rng, n=100000):
    
    x1 = rng.uniform(low = 0, high = 1, size = n)
    x2 = rng.uniform(low = 0, high = 1, size = n)
    x3 = rng.normal(loc = x1 + x2, scale = 0.1)
    y = rng.normal(loc=3*x1 + 2*x2 + 1, scale = 1)

    sim_data = pd.DataFrame({
        'x1' : x1,
        'x2' : x2,
        'x3' : x3,
        'y' : y
    })   

    return sim_data

train_data = gen_sim_data(rng)
train_data.head()

,x1,x2,x3,y
0,0.937212,0.357162,1.143867,3.758512
1,0.017030,0.799048,0.798812,3.978861
2,0.648718,0.353442,0.709564,1.626205
3,0.372858,0.863745,1.209894,2.650800
4,0.031801,0.608836,0.725445,2.854821


## **sygnet** pipeline

We follow a very similar pipeline to sci-kit learn: 

1. Instantiate a model:
   * Users must specify what type of GAN architecture to use: we recommend "wgan" for non-conditional synthetic data, and "cgan" when conditional labels will be supplied 
   * Optional arguments allow the user to customise the hidden layer structure, dropout proportions, layer norming, ReLU leakage, and whether to range match the final output
2. Fit the model to the training data
   * Users must supply the training data
   * Optional arguments allow the user to alter the default hyperparameters (epochs, learning rate, batch size etc.)
3. Sample from the trained model
   * Users must specify the number of synthetic observations to draw from the model
   * Optional arguments allow the user to control the format of the returned results, as well as to save the synthetic data to disk



## Basic example

In this first example, we set `mode = "wgan"` to use the Wassterstein GAN architecture. We fit the model to our simulated data, for a single epoch (real uses will require more epochs), and then generate 100 synthetic observations:

In [3]:
model = SygnetModel(mode = "wgan")
model.fit(data = train_data, epochs = 1)
synth_data = model.sample(nobs = 1000)

synth_data.head()

Epoch: 100%|██████████| 1/1 [00:08<00:00,  8.41s/it]


,x1,x2,x3,y
0,0.087701,-1.093142,0.0,0.727574
1,0.255873,-0.903843,0.0,0.503871
2,-0.186541,-1.124069,0.0,1.169112
3,-0.046919,-0.666187,0.0,0.670024
4,0.494258,-1.379985,0.0,0.566269


### GPU support

**sygnet** allows users to train the model using GPU computation, which should improve training times considerably. To run the synthetic generator on the GPU, we simply fit the model with the parameter `device = 'cuda'`. Here we are able to run 10 epochs in about as long as it takes to run 1 epoch on the CPU:

In [4]:
model_gpu = SygnetModel(mode = "wgan")
model_gpu.fit(data = train_data, epochs = 10, device='cuda')
synth_data = model_gpu.sample(nobs = 1000)

synth_data.head()

Epoch: 100%|██████████| 10/10 [00:09<00:00,  1.03it/s]


,x1,x2,x3,y
0,0.953159,0.284297,0.0,3.114364
1,0.745921,0.628582,0.0,3.176605
2,1.038066,-0.153269,0.0,2.768531
3,0.858559,0.822912,0.0,3.525120
4,0.552167,0.378724,0.0,2.439571


## Custom architectures

The above models are trained on the default parameters and for a short number of epochs. They are therefore not well-trained (as can be seen in the resulting synthetic data).

To improve our models, we can adjust the hyperparameter settings of our model. In this instance, we reduce the batch size and dropout proportion (relative to the default), increase the learning rate, and train for 50 epochs on the GPU:



In [5]:
model_custom = SygnetModel(mode = "wgan", dropout_p=0.1, mixed_activation=False)

model_custom.fit(
    train_data, 
    device = 'cuda', 
    epochs = 50, 
    batch_size=512,
    learning_rate=0.001
)

synth_data = model_custom.sample(1000)

Not using mixed activation function -- generated data may not conform to real data if it contains categorical columns.
Epoch: 100%|██████████| 50/50 [02:23<00:00,  2.88s/it]


To assess the quality of our model, we can see how well the synthetic data outcome variable `y` is modelled by the synthetic independent variables. To do so, we can regress `y` on `x1` and `x2`. Recall that in our DGP $$y \sim \mathcal{N}(\mu = 3\times X_1 + 2 \times X_2 + 1, \sigma).$$ Therefore, we should expect the coefficient vector to be close to `[3,2,1]`. 

In [8]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(synth_data.drop(['y','x3'], axis=1), synth_data['y'])

print(f"Synthetic coefficients = {[ '%.2f' % val for val in reg.coef_.tolist() + [reg.intercept_]]}")


Synthetic coefficients = ['2.96', '2.28', '0.94']


We find that, while the relationship is not perfectly captured, we nevertheless get data that resembles that relationship.